In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# Function to load and preprocess an image
def load_and_process_image(image_path, max_dim=512):
    img = Image.open(image_path)
    long = max(img.size)
    scale = max_dim / long
    img = img.resize((int(img.size[0] * scale), int(img.size[1] * scale)))
    img = np.expand_dims(np.array(img), axis=0)
    return tf.keras.applications.mobilenet_v2.preprocess_input(img)

# Function to deprocess an image
def deprocess_image(img):
    img = img[0]
    img = (img + 1.0) * 127.5  # Reverse MobileNetV2 preprocessing
    return np.clip(img, 0, 255).astype('uint8')

# Load content and style images
content_path = "content.jpg"  # Path to content image
style_path = "style.jpg"      # Path to style image

content_image = load_and_process_image(content_path)
style_image = load_and_process_image(style_path)

# Load pre-trained MobileNetV2 model
mobilenet = MobileNetV2(include_top=False, weights='imagenet')
mobilenet.trainable = False

# Define layers for content and style
content_layers = ['block_13_expand_relu']  # Adapted for MobileNetV2
style_layers = [
    'block_1_expand_relu', 'block_3_expand_relu', 
    'block_5_expand_relu', 'block_7_expand_relu', 
    'block_9_expand_relu'
]

def get_model():
    outputs = [mobilenet.get_layer(name).output for name in style_layers + content_layers]
    return Model([mobilenet.input], outputs)

# Define loss functions
def gram_matrix(input_tensor):
    channels = int(input_tensor.shape[-1])
    vectorized = tf.reshape(input_tensor, [-1, channels])
    return tf.matmul(vectorized, vectorized, transpose_a=True)

def compute_loss(outputs, content_target, style_target, style_weight=1e-2, content_weight=1e4):
    style_outputs, content_outputs = outputs[:-1], outputs[-1]
    style_loss = tf.add_n([tf.reduce_mean((gram_matrix(style) - gram_matrix(target))**2)
                           for style, target in zip(style_outputs, style_target)])
    content_loss = tf.reduce_mean((content_outputs - content_target)**2)
    return style_weight * style_loss + content_weight * content_loss

# Define the model and save it
extractor = get_model()
extractor.save('neural_style_transfer_model.h5')
print("Model saved as 'neural_style_transfer_model.h5'")

# Extract content and style features
content_target = extractor(content_image)[-1]
style_target = [gram_matrix(output) for output in extractor(style_image)[:-1]]

# Generate stylized image
generated_image = tf.Variable(content_image, dtype=tf.float32)

optimizer = tf.optimizers.Adam(learning_rate=5.0)
epochs = 100
for epoch in range(epochs):
    with tf.GradientTape() as tape:
        outputs = extractor(generated_image)
        loss = compute_loss(outputs, content_target, style_target)
    gradients = tape.gradient(loss, generated_image)
    optimizer.apply_gradients([(gradients, generated_image)])
    generated_image.assign(tf.clip_by_value(generated_image, -1.0, 1.0))  # Clip to MobileNetV2 range
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.numpy()}")

# Save and display results
styled_image = deprocess_image(generated_image.numpy())
plt.imshow(styled_image)
plt.axis('off')
plt.show()
